In [15]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers



In [49]:
import os
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY","pcsk_6jXPv3_RLghmrULaNQFxL4SPQSjxRQ4QpLwM8B4uyGeJdu5Np47SRfVKtgSyfqWLDGQauA")
PINECONE_ENV=os.environ.get("PINECONE_ENV","us-east-1")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["PINECONE_ENVIRONMENT"]=PINECONE_ENV

In [3]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)
    docs=loader.load()
    return docs

In [6]:
extracted=load_pdf("data/")

In [7]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted)
print(len(text_chunks))

40000


In [11]:
!pip install sentence_transformers


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: C:\Users\supri\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [45]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [46]:
embeddings=download_hugging_face_embeddings()

In [17]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [19]:
query_result=embeddings.embed_query("hello world")
print(len(query_result))

384


In [50]:
from pinecone import Pinecone

pc = Pinecone(api_key="PINECONE_API_KEY")  # replace with your real key

index_name="medical-chatbot"

In [51]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name="medical-chatbot",
)


In [55]:
docsearch=vectorstore.similarity_search("What are allergies?",k=3)  
print(docsearch[0].page_content)

‘‘What’s New in: Asthma and Allergic Rhinitis.’’Pulse
(September 20, 2004): 50.
Richard Robinson
Teresa G. Odle
Allergies
Definition
Allergies are abnormal reactions of the immune
system that occur in response to otherwise harmless
substances.
Description
Allergies are among the most common of medical
disorders. It is estimated that 60 million Americans, or
more than one in every five people, suffer from some
form of allergy, with similar proportions throughout


In [57]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}